# Mise à l'échelle des caractéristiques et taux d'apprentissage (Multi-variable)

## Objectifs
Dans ce TP, vous allez :
- Utiliser les fonctions à variables multiples développées dans le TP précédent
- Exécuter la descente de gradient sur un ensemble de données avec plusieurs caractéristiques
- Explorer l'impact du *taux d'apprentissage alpha* sur la descente de gradient
- Améliorer les performances de la descente de gradient par la *mise à l'échelle des caractéristiques* en utilisant la normalisation du score z.

## Outils
Vous utiliserez les fonctions développées dans le dernier TP ainsi que matplotlib et NumPy.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from lab_utils_multi import  load_house_data, run_gradient_descent 
from lab_utils_multi import  norm_plot, plt_equal_scale, plot_cost_i_w
from lab_utils_common import dlc
np.set_printoptions(precision=2)
plt.style.use('./deeplearning.mplstyle')

## Notation

|Notation générale <br />  | Description| Python (si applicable) |
|: ------------|: ------------------------------------------------------------||
| $a$ | scalaire, en non gras                                                      ||
| $\mathbf{a}$ | vecteur, en gras                                                 ||
| $\mathbf{A}$ | matrice, majuscule en gras                                         ||
| **Régression** |         |    |     |
|  $\mathbf{X}$ | matrice des exemples d'entraînement                  | `X_train` |   
|  $\mathbf{y}$  | cibles des exemples d'entraînement                | `y_train` 
|  $\mathbf{x}^{(i)}$, $y^{(i)}$ | $i_{ème}$ exemple d'entraînement | `X[i]`, `y[i]`|
| m | nombre d'exemples d'entraînement | `m`|
| n | nombre de caractéristiques dans chaque exemple | `n`|
|  $\mathbf{w}$  |  paramètre : poids,                       | `w`    |
|  $b$           |  paramètre : biais                                           | `b`    |     
| $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ | Le résultat de l'évaluation du modèle à  $\mathbf{x}^{(i)}$ paramétré par $\mathbf{w},b$: $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)}+b$  | `f_wb` | 
|$\frac{\partial J(\mathbf{w},b)}{\partial w_j}$| le gradient ou dérivée partielle du coût par rapport à un paramètre $w_j$ |`dj_dw[j]`| 
|$\frac{\partial J(\mathbf{w},b)}{\partial b}$| le gradient ou dérivée partielle du coût par rapport à un paramètre $b$| `dj_db`|

# Énoncé du problème

Comme dans les TPs précédents, vous utiliserez toujours l'exemple de la prédiction du prix des logements. L'ensemble de données d'entraînement contient de nombreux exemples avec 4 caractéristiques (taille, chambres, étages et âge) montrés dans le tableau ci-dessous. Notez que, dans ce TP, la caractéristique Taille est en pieds carrés alors que les TPs précédents utilisaient le millier de pieds carrés. Cet ensemble de données est plus grand que celui du TP précédent.

Nous aimerions construire un modèle de régression linéaire en utilisant ces valeurs afin que nous puissions ensuite prédire le prix pour d'autres maisons - disons, une maison de 1200 pieds carrés, 3 chambres, 1 étage, 40 ans.

## Ensemble de données : 
| Taille (pieds carrés) | Nombre de chambres  | Nombre d'étages | Âge de la maison | Prix (milliers de dollars)  |   
| ----------------| ------------------- |----------------- |--------------|----------------------- |  
| 952             | 2                   | 1                | 65           | 271.5                  |  
| 1244            | 3                   | 2                | 64           | 232                    |  
| 1947            | 3                   | 2                | 17           | 509.8                  |  
| ...             | ...                 | ...              | ...          | ...                    |


In [1]:
# charger l'ensemble de données
X_train, y_train = load_house_data()
X_features = ['taille(pieds carrés)','chambres','étages','âge']

NameError: name 'load_house_data' is not defined

Let's view the dataset and its features by plotting each feature versus price.

In [2]:
fig,ax=plt.subplots(1, 4, figsize=(12, 3), sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X_train[:,i],y_train)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("Prix (milliers de $)")
plt.show()

NameError: name 'plt' is not defined

La représentation graphique de chaque caractéristique par rapport à la cible, le prix, donne une indication de quelles caractéristiques ont la plus forte influence sur le prix. Ci-dessus, l'augmentation de la taille augmente également le prix. Les chambres et les étages ne semblent pas avoir un fort impact sur le prix. Les maisons plus récentes ont des prix plus élevés que les maisons plus anciennes.

<a name="toc_15456_5"></a>
## Descente de gradient avec plusieurs variables
Voici les équations que vous avez développées dans le dernier TP sur la descente de gradient pour plusieurs variables :

$$\begin{align*} \text{répéter}&\text{ jusqu'à convergence:} \; \lbrace \newline\;
& w_j := w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{pour j = 0..n-1}\newline
&b\ \ := b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}  \newline \rbrace
\end{align*}$$

où, n est le nombre de caractéristiques, les paramètres $w_j$,  $b$, sont mis à jour simultanément et où  

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{2}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{3}
\end{align}
$$
* m est le nombre d'exemples d'entraînement dans l'ensemble de données

*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ est la prédiction du modèle, tandis que $y^{(i)}$ est la valeur cible

## Taux d'apprentissage

Le cours a porté sur certaines questions liées à la définition du taux d'apprentissage $\alpha$. Le taux d'apprentissage contrôle la taille de la mise à jour des paramètres. Voir l'équation (1) ci-dessus. Il est commun à tous les paramètres.

Exécutons la descente de gradient et essayons quelques paramétrages de $\alpha$ sur notre ensemble de données.

### $\alpha$ = 9.9e-7

In [3]:
#On initialise alpha à 9.9e-7
_, _, hist = run_gradient_descent(X_train, y_train, 10, alpha = 9.9e-7)

NameError: name 'run_gradient_descent' is not defined

It appears the learning rate is too high.  The solution does not converge. Cost is *increasing* rather than decreasing. Let's plot the result:

In [ ]:
plot_cost_i_w(X_train, y_train, hist)

Le graphique à droite montre la valeur de l'un des paramètres, $w_0$. À chaque itération, il dépasse la valeur optimale et, par conséquent, le coût finit par augmenter plutôt que de se rapprocher du minimum. Notez que ce n'est pas une image complètement précise car il y a 4 paramètres qui sont modifiés à chaque passage plutôt qu'un seul. Ce graphique ne montre que $w_0$ avec les autres paramètres fixés à des valeurs très petites. Dans ce graphique et les suivants, vous pouvez remarquer que les lignes bleue et orange sont légèrement décalées.


### $\alpha$ = 9e-7
Essayons une valeur un peu plus petite et voyons ce qui se passe.

In [ ]:
# On initialise alpha à 9e-7
_,_,hist = run_gradient_descent(X_train, y_train, 10, alpha = 9e-7)

Le coût diminue au fur et à mesure, montrant que alpha n'est pas trop grand. 

In [ ]:
plot_cost_i_w(X_train, y_train, hist)

Sur la gauche, vous voyez que le coût diminue comme il se doit. Sur la droite, vous pouvez voir que $w_0$ oscille toujours autour du minimum, mais il diminue à chaque itération plutôt qu'augmenter. Notez ci-dessus que `dj_dw[0]` change de signe à chaque itération lorsque `w[0]` passe au dessus la valeur optimale.
Cette valeur alpha convergera. Vous pouvez varier le nombre d'itérations pour voir comment elle se comporte.

### $\alpha$ = 1e-7
Essayons une valeur plus petite d'alpha et voyons ce qui se passe.

In [ ]:
# on initialise alpha à 1e-7
_,_,hist = run_gradient_descent(X_train, y_train, 10, alpha = 1e-7)

Le coût diminue au fur et à mesure, montrant que alpha n'est pas trop grand. 

In [ ]:
plot_cost_i_w(X_train,y_train,hist)


Sur la gauche, vous voyez que le coût diminue comme il se doit. Sur la droite, vous pouvez voir que $w_0$ diminue sans traverser le minimum. Notez ci-dessus que `dj_w0` est négatif tout au long de l'exécution. Cette solution convergera également, bien que pas tout à fait aussi rapidement que l'exemple précédent.

## Mise à l'échelle des caractéristiques

Pendant les cours, l'importance de la remise à l'échelle de l'ensemble de données afin que les caractéristiques aient une plage similaire a été mentionnée.
Si vous êtes intéressé par les détails de pourquoi c'est le cas, cliquez sur l'en-tête 'détails' ci-dessous. Sinon, la section ci-dessous vous guidera à travers une mise en œuvre de comment faire la mise à l'échelle des caractéristiques.

<details>
<summary>
    <font size='3', color='darkgreen'><b>Détails</b></font>
</summary>

Revenons à la situation avec $\alpha$ = 9e-7. C'est assez proche de la valeur maximale que nous pouvons définir pour $\alpha$ sans diverger. Voici une courte exécution montrant les premières itérations :

<figure>
    <img src="./images/C1_W2_Lab06_ShortRun.PNG" style="width:1200px;" >
</figure>

Ci-dessus, bien que le coût soit en train de diminuer, il est clair que $w_0$ progresse plus rapidement que les autres paramètres en raison de son gradient beaucoup plus grand.

Le graphique ci-dessous montre le résultat d'une très longue exécution avec $\alpha$ = 9e-7. Cela prend plusieurs heures.

<figure>
    <img src="./images/C1_W2_Lab06_LongRun.PNG" style="width:1200px;" >
</figure>
    
Ci-dessus, vous pouvez voir que le coût a diminué lentement après sa réduction initiale. Remarquez la différence entre `w0` et `w1`,`w2`,`w3` ainsi que `dj_dw0` et `dj_dw1-3`. `w0` atteint sa valeur finale presque très rapidement et `dj_dw0` a rapidement diminué à une petite valeur montrant que `w0` est proche de la valeur finale. Les autres paramètres ont été réduits beaucoup plus lentement.

Pourquoi cela ? Y a-t-il quelque chose que nous pouvons améliorer ?
<figure>
    <center> <img src="./images/C1_W2_Lab06_scale.PNG"   ></center>
</figure>   

La figure ci-dessus montre pourquoi les $w$ sont mis à jour de manière inégale. 
- $\alpha$ est partagé par toutes les mises à jour de paramètres ($w$ et $b$).
- le terme d'erreur commun est multiplié par les caractéristiques pour les $w$. (pas $b$).
- les caractéristiques varient significativement en magnitude, ce qui fait que certaines caractéristiques se mettent à jour beaucoup plus rapidement que d'autres. Dans ce cas, $w_0$ est multiplié par 'taille(pieds carrés)', qui est généralement > 1000, tandis que $w_1$ est multiplié par 'nombre de chambres', qui est généralement de 2-4. 
    
La solution est la mise à l'échelle des caractéristiques.

Dans le cours, trois techniques différentes ont été discutées : 
- La mise à l'échelle des caractéristiques, qui consiste essentiellement à diviser chaque caractéristique positive par sa valeur maximale, ou plus généralement, à redimensionner chaque caractéristique par ses valeurs minimale et maximale en utilisant (x-min)/(max-min). Les deux méthodes normalisent les caractéristiques à l'intervalle de -1 et 1, la première méthode fonctionnant pour les caractéristiques positives, ce qui est simple et convient bien pour l'exemple de la conférence, et la seconde méthode fonctionnant pour toutes les caractéristiques.
- Normalisation de la moyenne : $x_i := \dfrac{x_i - \mu_i}{max - min} $ 
- Normalisation du score Z que nous explorerons ci-dessous.

### Normalisation par le score z
Après la normalisation par le score z, toutes les caractéristiques auront une moyenne de 0 et un écart type de 1.

Pour mettre en œuvre la normalisation par le score z, ajustez vos valeurs d'entrée comme le montre cette formule :
$$x^{(i)}_j = \dfrac{x^{(i)}_j - \mu_j}{\sigma_j} \tag{4}$$ 
où $j$ sélectionne une caractéristique ou une colonne dans la matrice $\mathbf{X}$. $µ_j$ est la moyenne de toutes les valeurs pour la caractéristique (j) et $\sigma_j$ est l'écart type de la caractéristique (j).
$$
\begin{align}
\mu_j &= \frac{1}{m} \sum_{i=0}^{m-1} x^{(i)}_j \tag{5}\\
\sigma^2_j &= \frac{1}{m} \sum_{i=0}^{m-1} (x^{(i)}_j - \mu_j)^2  \tag{6}
\end{align}
$$

>**Note d'implémentation :** Lors de la normalisation des caractéristiques, il est important
de stocker les valeurs utilisées pour la normalisation - la valeur moyenne et l'écart type utilisés pour les calculs. Après avoir appris les paramètres
du modèle, nous voulons souvent prédire les prix des maisons que nous n'avons pas
vu auparavant. Étant donné une nouvelle valeur x (surface du salon et nombre de chambres),
nous devons d'abord normaliser x en utilisant la moyenne et l'écart type
que nous avions précédemment calculé à partir de l'ensemble d'entraînement.

**Implémentation**
Dans la cellule ci-dessous, implémentez la fonction "zscore_normalize_features" en suivant les formules ci-dessus.

In [4]:
def zscore_normalize_features(X):
    """
    calcule X, normalisé par le score z par colonne
    
    Args:
      X (ndarray (m,n))     : données d'entrée, m exemples, n caractéristiques
      
    Returns:
      X_norm (ndarray (m,n)): entrée normalisée par colonne
      mu (ndarray (n,))     : moyenne de chaque caractéristique
      sigma (ndarray (n,))  : écart type de chaque caractéristique
    """

# INDICE
# si vous avez implémenté la fonction correctement, après l'avoir exécutée 
# comme "X_norm, X_mu, X_sigma = zscore_normalize_features(X_train)" vous devriez obtenir :
# X_mu = [1.42e+03 2.72e+00 1.38e+00 3.84e+01], 
# X_sigma = [411.62   0.65   0.49  25.78]
    mu = np.mean(X, axis =0)
    sigma = np.std(X, axis=0)
    X_norm = (X-mu) / sigma
    return (X_norm, mu, sigma)
 
# vérifions notre travail
# from sklearn.preprocessing import scale
# scale(X_orig, axis=0, with_mean=True, with_std=True, copy=True)



Regardons les étapes impliquées dans la normalisation par le score Z. Le graphique ci-dessous montre la transformation étape par étape.

In [5]:
mu     = np.mean(X_train,axis=0)   
sigma  = np.std(X_train,axis=0) 
X_mean = (X_train - mu)
X_norm = (X_train - mu)/sigma      

fig,ax=plt.subplots(1, 3, figsize=(12, 3))
ax[0].scatter(X_train[:,0], X_train[:,3])
ax[0].set_xlabel(X_features[0]); ax[0].set_ylabel(X_features[3]);
ax[0].set_title("non normalisé")
ax[0].axis('equal')

ax[1].scatter(X_mean[:,0], X_mean[:,3])
ax[1].set_xlabel(X_features[0]); ax[0].set_ylabel(X_features[3]);
ax[1].set_title(r"X - $\mu$")
ax[1].axis('equal')

ax[2].scatter(X_norm[:,0], X_norm[:,3])
ax[2].set_xlabel(X_features[0]); ax[0].set_ylabel(X_features[3]);
ax[2].set_title(r"Normalisé par le score Z")
ax[2].axis('equal')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle("distribution des caractéristiques avant, pendant, après normalisation")
plt.show()

NameError: name 'np' is not defined

Le graphique ci-dessus montre la relation entre deux des paramètres de l'ensemble d'entraînement, "age" et "taille (pied carrés)". *Ces derniers sont tracés à l'échelle égale*. 
- À gauche : Non normalisé : La plage de valeurs ou la variance de la caractéristique 'size(sqft)' est beaucoup plus grande que celle de l'âge
- Au milieu : La première étape supprime la valeur moyenne ou moyenne de chaque caractéristique. Cela laisse des caractéristiques qui sont centrées autour de zéro. Il est difficile de voir la différence pour la caractéristique 'âge', mais 'size(sqft)' est clairement autour de zéro.
- À droite : La deuxième étape divise par l'écart type. Cela laisse les deux caractéristiques centrées à zéro avec une échelle similaire.

Normalisons les données et comparons-les aux données originales.

In [6]:
# On normalise les données originales
X_norm, X_mu, X_sigma = zscore_normalize_features(X_train)
print(f"X_mu = {X_mu}, \nX_sigma = {X_sigma}")
print(f"Plage de valeurs par colonne dans X original : {np.ptp(X_train,axis=0)}")   
print(f"Plage de valeurs par colonne dans X normalisé :{np.ptp(X_norm,axis=0)}")

NameError: name 'X_train' is not defined

La plage de valeurs maximale de chaque colonne est réduite d'un facteur de milliers à un facteur de 2-3 par la normalisation.

In [7]:
fig,ax=plt.subplots(1, 4, figsize=(12, 3))
for i in range(len(ax)):
    norm_plot(ax[i],X_train[:,i],)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("compte");
fig.suptitle("distribution des caractéristiques avant la normalisation")
plt.show()
fig,ax=plt.subplots(1,4,figsize=(12,3))
for i in range(len(ax)):
    norm_plot(ax[i],X_norm[:,i],)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("compte"); 
fig.suptitle("distribution des caractéristiques après la normalisation")

plt.show()

NameError: name 'plt' is not defined

Remarquez, ci-dessus, que la plage des données normalisées (axe des x) est centrée autour de zéro et est approximativement de +/- 2. Le plus important, c'est que la plage est similaire pour chaque caractéristique.

Relançons notre algorithme de descente de gradient avec des données normalisées.
Notez la **valeur beaucoup plus grande d'alpha**. Cela accélérera la descente de gradient.

In [ ]:
w_norm, b_norm, hist = run_gradient_descent(X_norm, y_train, 1000, 1.0e-1, )

Les caractéristiques mises à l'échelle obtiennent des résultats très précis **beaucoup, beaucoup plus rapidement !**. Remarquez que le gradient de chaque paramètre est minuscule à la fin de cette exécution assez courte. Un taux d'apprentissage de 0,1 est un bon départ pour la régression avec des caractéristiques normalisées.
Traçons nos prédictions par rapport aux valeurs cibles. Notez que la prédiction est faite en utilisant la caractéristique normalisée tandis que le graphique est montré en utilisant les valeurs de caractéristiques originales.

In [8]:
# Prédire la cible en utilisant des caractéristiques normalisées
m = X_norm.shape[0]
yp = np.zeros(m)
for i in range(m):
    yp[i] = np.dot(X_norm[i], w_norm) + b_norm

# Tracer les prédictions et les cibles par rapport aux caractéristiques originales    
fig,ax=plt.subplots(1,4,figsize=(12, 3),sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X_train[:,i],y_train, label = 'cible')
    ax[i].set_xlabel(X_features[i])
    ax[i].scatter(X_train[:,i],yp,color=dlc["dlorange"], label = 'prédiction')
ax[0].set_ylabel("Prix"); ax[0].legend();
fig.suptitle("cible versus prédiction en utilisant le modèle normalisé par le score Z")
plt.show()

NameError: name 'X_norm' is not defined

Les résultats semblent bons. Quelques points à noter :
- avec plusieurs caractéristiques, nous ne pouvons plus avoir un seul graphique montrant les résultats par rapport aux caractéristiques.
- lors de la génération du graphique, les caractéristiques normalisées ont été utilisées. Toute prédiction utilisant les paramètres appris à partir d'un ensemble d'entraînement normalisé doit également être normalisée.

**Prédiction**
L'objectif de générer notre modèle est de l'utiliser pour prédire les prix des logements qui ne sont pas dans l'ensemble de données. Prédisons le prix d'une maison de 1200 pieds carrés, 3 chambres, 1 étage, 40 ans. Rappelez-vous, vous devez normaliser les données avec la moyenne et l'écart type dérivés lorsque les données d'entraînement ont été normalisées.

In [9]:
# Tout d'abord, normalisons notre exemple.
x_house = np.array([1200, 3, 1, 40])
x_house_norm = (x_house - X_mu) / X_sigma
print(x_house_norm)
x_house_predict = np.dot(x_house_norm, w_norm) + b_norm
print(f" prix prédit d'une maison de 1200 pieds carrés, 3 chambres, 1 étage, 40 ans = ${x_house_predict*1000:0.0f}")

NameError: name 'np' is not defined

**Contours de coût**

Une autre façon de voir la mise à l'échelle des caractéristiques est en termes de contours de coût. Lorsque les échelles des caractéristiques ne correspondent pas, le tracé du coût par rapport aux paramètres dans un tracé de contour est asymétrique.

Dans le tracé ci-dessous, l'échelle des paramètres est adaptée. Le tracé de gauche est le tracé de contour de coût de w[0], les pieds carrés par rapport à w[1], le nombre de chambres avant la normalisation des caractéristiques. Le tracé est si asymétrique que les courbes complétant les contours ne sont pas visibles. En revanche, lorsque les caractéristiques sont normalisées, le contour de coût est beaucoup plus symétrique. Le résultat est que les mises à jour des paramètres lors de la descente de gradient peuvent faire des progrès égaux pour chaque paramètre.

In [ ]:
plt_equal_scale(X_train, X_norm, y_train)

## Congratulations!
In this lab you:
- utilized the routines for linear regression with multiple features you developed in previous labs
- explored the impact of the learning rate  $\alpha$ on convergence 
- discovered the value of feature scaling using z-score normalization in speeding convergence

## Félicitations !
Dans ce TP, vous avez :
- utilisé les fonctions pour la régression linéaire avec plusieurs caractéristiques que vous avez développées dans les TPs précédents
- exploré l'impact du taux d'apprentissage $\alpha$ sur la convergence
- découvert l'intérêt de la mise à l'échelle des caractéristiques en utilisant la normalisation par le score z pour accélérer la convergence

## Remerciements
Les données sur le logement proviennent de l'[ensemble de données sur le logement d'Ames](http://jse.amstat.org/v19n3/decock.pdf) compilé par Dean De Cock pour être utilisé dans l'éducation en science des données.